In [ ]:

from torch.utils.data import DataLoader
from statsmodels.nonparametric.smoothers_lowess import lowess
import torch 
import gc
import plotly.graph_objects as go
from torch import nn 
from torch.utils.data import Dataset
from torchvision.transforms.v2 import PILToTensor,Compose
from sklearn.model_selection import train_test_split
from datetime import datetime
import re
from torch.utils.tensorboard import SummaryWriter
import torchvision
import pandas as pd
import math
import os
from tqdm import tqdm
from einops import rearrange 
import pickle
import matplotlib.pyplot as plt 
from functools import partial, reduce
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np
import seaborn as sns
import matplotlib

import panel as pn

import holoviews as hv

import random
import math
import time
import random
from yellowbrick.text import TSNEVisualizer
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

from holoviews import dim, opts

hv.extension("plotly")
pn.extension("plotly")
pn.extension('tabulator', theme='dark')
pn.config.theme = 'dark'
hv.renderer('plotly').theme = 'dark'
import sys

sys.path.append(os.path.abspath(os.path.join('..')))
from src.models.unified_classifier import ClassifierModelPipeline, evaluate_unified_classifier_model, predictDenoise, predictCLS, ClassificationHeadDummy
from src.models.conv_classifier import ConvChromoClassifier, ConvPredictCLS
from src.models.denoiser import DenoiserModelPipeline, DenoiserModel, get_forward_diffusion_params, forward_add_noise
from src.md import MDDataset, generate_phases_for_dense_validation, MDDenseSet, MDLoadable, MDDense, MDDensePhaseData
from src.preprocessing import ChromoDataContainer, ChromoData


device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# This notebook loads all trained classification models, evaluate them against dense MD and RD 
# and saves the results to a pickle file for future parsing and visualization

# Loading DenseRD
chrom_data = ChromoDataContainer.load_from_pkl()

# TODO: Split MD and DenseD to separatte notebooks
# 1) MD generates source for Dense MD
# 2) DenseD generates Both Dense MD and Dense RD
distiled_real_mu = generate_phases_for_dense_validation(chrom_data.gausians, do_real_mu=True)
distiled_real_lowess = generate_phases_for_dense_validation(chrom_data.gausians, do_real_mu=False)
distiled_mock_mu = MDLoadable.load(
    config_name="dataset_128_128_784_classifier_control_set_mu",
    config_folder="../configs/MDDense",
    dataset_folder="../data/MDDense",
    dataset_cls=MDDenseSet)
print(f"Len of distiled_mock_mu: {len(distiled_mock_mu)}")
distiled_mock_lowess = MDLoadable.load(
    config_name="dataset_128_128_784_classifier_control_set_lowess",
    config_folder="../configs/MDDense",
    dataset_folder="../data/MDDense",
    dataset_cls=MDDenseSet)
print(f"Len of distiled_mock_lowess: {len(distiled_mock_lowess)}")

In [ ]:
# this is a test of a BOOST algorithm from a paper "SHOREmap v3.0: Fast and Accurate Identifi cation of Causal Mutations from Forward Genetic Screens"

def calculate_boost_value(probabilities, window_size=50):

    def boost_value(obs_theta, target_theta=0.997):
        return 1 / abs(1 - max(target_theta, 1-target_theta) / 
                      max(obs_theta, 1-obs_theta))
    
    # Calculate mean probabilities in sliding windows
    theta_obs = np.array([np.mean(probabilities[i:i+window_size]) 
                          for i in range(len(probabilities) - window_size)])
    
    # Calculate boost values for each window
    boost_values = np.array([boost_value(theta) for theta in theta_obs])
    
    # Find peak location (adding window_size//2 to account for convolution)
    # peak_location = np.argmax(boost_values) + window_size//2
    
    peak_location = np.argmax(boost_values)
    print(f"Peak location: {peak_location}, len of prob: {len(probabilities)}, window_size: {window_size}, len of boost_values: {len(boost_values)}, len of theta_obs: {len(theta_obs)}")
    peak_location = peak_location + window_size//2
    
    return boost_values, peak_location

real_mus = []
for record in chrom_data.gausians:
    if record.m_index:
        real_mus.append(record)

for record in real_mus:
    print("=====================================")
    probabilities = record.array['Gxx_ratio'].values
    boost_values, peak_location = calculate_boost_value(probabilities, window_size=500)
    
    pos_peak = record.array['POS'].iloc[peak_location]
    pos_real = record.array['POS'].iloc[record.m_index]
    accuracy = 100 * (1 - abs(peak_location-record.m_index) / len(probabilities))
    
    print(f"G number: {record.g_number}")
    print(f"Peak location: {peak_location}")
    print(f"POS on boost location: {pos_peak}")
    print(f"Real POS: {pos_real}")
    print(f"Delta MB: {abs(pos_peak-pos_real)/1e6:.3f}")
    print(f"Accuracy %: {accuracy}")

In [ ]:
# Automated evaluator for Unified classifier with Linear and Attention heads

configs_dir = '../configs/classifier'
configs = os.listdir(configs_dir)

print(f"Found {len(configs)} configs")
for config in configs:
    print(config)
    
pipelines = []

for config in tqdm(configs):
    print("=====================================")
    classifierModelPipeline = ClassifierModelPipeline.load(config, skip_data_load=False)
    pipelines.append(classifierModelPipeline)
    classifierModelPipeline.model.to(device)
    classifierModelPipeline.model.eval()
    print(f"Loaded model {classifierModelPipeline.config.model.description}")
    print(f"Loaded model {config}")
    
print("Done loading models")

evaluation_results = []

T, betas, alphas, alphas_cumprod, alphas_cumprod_prev, variance = get_forward_diffusion_params()
alphas_cumprod = alphas_cumprod.to(device)

for pipeline in tqdm(pipelines):
    result_val = evaluate_unified_classifier_model(
                    pipeline.model,
                    (
                        distiled_real_mu, 
                        distiled_real_lowess,
                        distiled_mock_mu.data, 
                        distiled_mock_lowess.data
                    ),
                    alphas_cumprod,
                    pipeline.config.training.timestep,
                    pipeline.config.training.label_num,
                    pipeline.config.training.label_count,
                    pipeline.config.training.batch_size,
                    predictCLS  
                )
    evaluation_results.append((pipeline.config.model.description, result_val, pipeline.config, pipeline.denoiser_model.config))

In [ ]:
# Evaluate Trained Conv Classifier

conv_classifier_dict = torch.load("../checkpoints/classifier/conv_classifier.pth")
model = ConvChromoClassifier(seq_len=784, class_len=784)
model.load_state_dict(conv_classifier_dict)
model.to(device)
model.eval()

result_val = evaluate_unified_classifier_model(
    model,
    (
        distiled_real_mu, 
        distiled_real_lowess,
        distiled_mock_mu.data, 
        distiled_mock_lowess.data
    ),
    None,
    None,
    None,
    768,
    512,
    ConvPredictCLS  
)

evaluation_results.append(("Conventional separate classifier", result_val, None, None))

In [ ]:
# Evaluate zero shot unified classifier
# load classifier, replace head with dummy head and run classification on cls 2.0
# Method is N**2, may take 15mins to run, comment out whole cell if not needed

classifier2ModelPipeline = ClassifierModelPipeline.load("unified_cls_attn_full_labels_denoiser_conv_4", skip_data_load=False)
# replace with empty CLS head, predictDenoise method will do the job instead of it
classifier2ModelPipeline.model.classifier_head = ClassificationHeadDummy()
classifier2ModelPipeline.model.to(device)
classifier2ModelPipeline.model.eval()
result_val = evaluate_unified_classifier_model(
                    classifier2ModelPipeline.model,
                    (
                        distiled_real_mu, 
                        distiled_real_lowess,
                        distiled_mock_mu.data, 
                        distiled_mock_lowess.data
                    ),
                    alphas_cumprod,
                    450, # empirically, the best timestep for predictDenoise algorithm
                    pipeline.config.training.label_num,
                    pipeline.config.training.label_count,
                    pipeline.config.training.batch_size,
                    predictDenoise  
                )
# remove row with classifier 2.0
evaluation_results = [x for x in evaluation_results if x[0] != "Zero-shot unified classifier"]
evaluation_results.append(("Zero-shot unified classifier", result_val, None, None))

In [7]:
# Save results to pickle file
os.makedirs("../metrics/results", exist_ok=True)
# get last timestamp
timestamp_int = int(datetime.now().timestamp())
result_filename = f"../metrics/results/{timestamp_int}_results.pkl"
with open(result_filename, 'wb') as f:
    pickle.dump(evaluation_results, f)